# 08. Azure AI Document Intelligence - Invoice Model

> https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/overview?view=doc-intel-4.0.0

## A. Create an AI Document Intelligence resource and set up environment to run notebook

**_Prerequsite_:** <br>

**AI Document Intelligence resource**: <br>
To create a AI Document Intelligence resource in your Azure subscription:
Please follow the steps as specified https://learn.microsoft.com/en-us/azure/ai-services/document-intelligence/create-document-intelligence-resource?view=doc-intel-4.0.0


Get your newly created Document Intelligence service in the Azure portal and on the **Keys and Endpoint** page, copy the **Key1** and **Endpoint** values and paste them in the code cell below, replacing **YOUR_FORM_KEY** and **YOUR_FORM_ENDPOINT**.

**Environment: **

1. **AML workspace**: Please ensure you have Python 3.10 version or above ie select **Python 3.10 - SDK v2** as kernel in AML Notebook. <br>
2. **VS Code**: Please select Python **Python 3.10** above or try to set up virtual envi by following steps https://code.visualstudio.com/docs/python/environments

Please skip this step if you are already done it

## B. Install AI Doc Intelligence library

In [ ]:
# install azure-ai-formrecognizer python library and restart the kernal after installation
# ignore this step is you have already installed azure-ai-formrecognizer python library
# %pip install azure-ai-formrecognizer --upgrade --user

## C. Setting up AI Doc Intelligence endpoint and key

In [2]:
# Import the os module for interacting with the operating system
import os
# Import for handling resource not found errors
from azure.core.exceptions import ResourceNotFoundError
# Import for authenticating with the Azure service
from azure.core.credentials import AzureKeyCredential
# Import for analysing forms
from azure.ai.formrecognizer import DocumentAnalysisClient, AnalyzeResult


# load the environments details
from dotenv import load_dotenv
load_dotenv()

# Set `<your-endpoint>` and `<your-key>` variables with the values from the Azure portal
# END_POINT is the endpoint URL of your AI Document Intelligence service
# END_POINT_KEY is the key for your AI Document Intelligence service
END_POINT = os.getenv("FORM_RECOGNIZER_ENDPOINT")
END_POINT_KEY = os.getenv("FORM_RECOGNIZER_KEY")

# Create a DocumentAnalysisClient instance
# This client is used to interact with the Azure Form Recognizer service
# It is initialized with your endpoint and key
form_recognizer_client = DocumentAnalysisClient(END_POINT, AzureKeyCredential(END_POINT_KEY))

## D. Extract the Invoice Document

In [3]:
# Define the URL of the sample invoice document to analyze
# You can change the URL pointing to your sample invoice docs but ensure you provide appropriate access
invoiceUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-invoice.pdf"

# Start the analysis of the invoice document using the prebuilt invoice model
# The result is a poller object that can be used to check the status of the operation
poller = form_recognizer_client.begin_analyze_document_from_url("prebuilt-invoice",invoiceUrl)

# Print the poller object. This is useful for debugging purposes to ensure the analysis has started correctly.
print(poller)

# Get the result of the analysis
result = poller.result()

# Print the documents in the result. This will display the analyzed information from the invoice document.
print(result.documents)

[AnalyzedDocument(doc_type=invoice, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.0, y=0.0), Point(x=8.5, y=0.0), Point(x=8.5, y=11.0), Point(x=0.0, y=11.0)])], spans=[DocumentSpan(offset=0, length=748)], fields={'AmountDue': DocumentField(value_type=currency, value=CurrencyValue(amount=610.0, symbol=$, code=USD), content=$610.00, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=7.3587, y=7.8012), Point(x=7.9174, y=7.8012), Point(x=7.9222, y=7.9731), Point(x=7.3539, y=7.9731)])], spans=[DocumentSpan(offset=653, length=7)], confidence=0.95), 'BillingAddress': DocumentField(value_type=address, value=AddressValue(house_number=123, po_box=None, road=Bill St, city=Redmond, state=WA, postal_code=98052, country_region=None, street_address=123 Bill St, unit=None, city_district=None, state_district=None, suburb=None, house=None, level=None), content=123 Bill St,
Redmond WA, 98052, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=0.5635, y=4.3

## E. Extracted Layout Document insights/ response as a JSON format 

In [4]:
# Import necessary libraries
# json for handling JSON data
# datetime and time for generating timestamps
# AzureJSONEncoder for serializing Python objects to JSON
# urlparse for parsing URLs
import json
import datetime
import time
from azure.core.serialization import AzureJSONEncoder
from urllib.parse import urlparse


# Generate a unique filename based on the current timestamp and the basename of the URL
filename = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d%H%M%S')+"_"+os.path.splitext(os.path.basename(urlparse(invoiceUrl).path))[0]

# Convert the result of the analysis (which is a model) to a dictionary
analyze_result_dict = result.to_dict()

# Open a new file with the generated filename and write the dictionary to it as JSON
# Use the AzureJSONEncoder to handle any types that aren't serializable by default
with open(str(filename), 'w') as f:
        json.dump(analyze_result_dict, f, cls=AzureJSONEncoder,indent=4)

# Convert the dictionary back to the original model
model = AnalyzeResult.from_dict(analyze_result_dict)

# Print some information about the model
print("--------------JSON Response from Model Starts---------------------")
# use the model as normal
print("Model ID: '{}'".format(model.model_id))
print("Number of pages analyzed {}".format(len(model.pages)))
print("API version used: {}".format(model.api_version))

# Print the dictionary (which is the JSON response from the model)
print(json.dumps(analyze_result_dict,cls=AzureJSONEncoder,indent=4))
print("--------------JSON Response from Model Ends---------------------")

--------------JSON Response from Model Starts---------------------
Model ID: 'prebuilt-invoice'
Number of pages analyzed 1
API version used: 2023-07-31
{
    "api_version": "2023-07-31",
    "model_id": "prebuilt-invoice",
    "content": "CONTOSO LTD.\nINVOICE\nContoso Headquarters\nINVOICE: INV-100\n123 456th St\nDATE: 11/15/2019\nNew York, NY, 10001\nDUE DATE: 12/15/2019\nCUSTOMER NAME: MICROSOFT CORPORATION\nCUSTOMER ID: CID-12345\nMicrosoft Corp\n123 Other St,\nRedmond WA, 98052\nBILL TO:\nSHIP TO:\nSERVICE ADDRESS:\nMicrosoft Finance\nMicrosoft Delivery\nMicrosoft Services\n123 Bill St,\n123 Ship St,\n123 Service St,\nRedmond WA, 98052\nRedmond WA, 98052\nRedmond WA, 98052\nSALESPERSON\nP.O. NUMBER\nREQUISITIONER\nSHIPPED VIA\nF.O.B. POINT\nTERMS\nPO-3333\nQUANTITY\nDESCRIPTION\nUNIT PRICE\nTOTAL\n1\nTest for 23 fields\n1\n$100.00\nSUBTOTAL\n$100.00\nSALES TAX\n$10.00\nTOTAL\n$110.00\nPREVIOUS BALANCE\n$500.00\nTOTAL DUE\n$610.00\nTHANK YOU FOR YOUR BUSINESS!\nREMIT TO:\nContoso B